In [1]:
import os
import rbc

In [2]:
workdir = '/Users/rchales/data/regional-bias-correction/colombia-magdalena'
drain_shape = os.path.join(workdir, 'gis_inputs', 'magdalena_drainlines_namedattrs.json')
hist_sim_nc = os.path.join(workdir, 'data_simulated', 'south_america_era5_qout.nc')

In [3]:
assign_table = rbc.table.read(workdir)

In [4]:
a = assign_table.copy()

In [5]:
sorted(set(a['sim-fdc-cluster'].values))

[0, 1, 2, 3]

In [6]:
b = a[a['sim-fdc-cluster'] == 0]

In [7]:
b.to_csv('tmp.csv')

In [8]:
b

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster
16,9000433,9000346,73937587.8,1,NaN,NaN,NaN,0,NaN
17,9000433,9000347,103358605.7,1,NaN,NaN,NaN,0,NaN
29,9000348,9000391,820711818.1,3,NaN,NaN,NaN,0,NaN
43,9000469,9000432,87711889.9,1,NaN,NaN,NaN,0,NaN
44,9000391,9000435,378978505.3,2,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...
2224,9009844,9009936,531307435.9,2,21257070.0,21257070.0,gauged,0,3.0
2232,9010320,9010327,337199911.3,1,21257090.0,21257090.0,gauged,0,0.0
2240,9011017,9011229,327856283.4,2,21227010.0,21227010.0,gauged,0,2.0
2278,9013769,9014132,176847798.1,1,22057050.0,22057050.0,gauged,0,2.0


In [9]:
sorted(set(b['stream_order']))

[1, 2, 3]

In [10]:
c = b[b['stream_order'] == 3]

In [11]:
c

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster
29,9000348,9000391,8.207118e+08,3,NaN,NaN,NaN,0,NaN
218,9001222,9001219,1.279048e+09,3,NaN,9001336.0,propagation-upstream-1,0,NaN
219,9001243,9001222,1.391088e+09,3,NaN,9001336.0,propagation-upstream-2,0,NaN
1105,9006357,9006694,1.634095e+09,3,NaN,9006357.0,propagation-upstream-3,0,NaN


In [12]:
c[c['obs-fdc-cluster'] == 3]

,downstream_model_id,model_id,drainage_area_mod,stream_order,gauge_id,assigned_id,reason,sim-fdc-cluster,obs-fdc-cluster


In [13]:
rbc.gis.clip_by_ids(workdir, c['model_id'].values, drain_shape)

In [14]:
rbc.gis.clip_by_ids(workdir, [9001336, 9006357], drain_shape, 'gauges')

In [ ]:
# # Only need to do this step 1x ever
# rbc.prep.scaffold_working_directory(workdir)

In [ ]:
# Create the gauge_table and drain_table.csv
# Scripts not provided, check readme for instructions

In [ ]:
# Generate the assignments table
assign_table = rbc.prep.gen_assignments_table(workdir)
rbc.table.cache(workdir, assign_table)

# or read the existing assignments table
assign_table = rbc.table.read(workdir)

In [ ]:
# Prepare the observation and simulation data
# Only need to do this step 1x ever
rbc.prep.historical_simulation(hist_sim_nc, workdir)
rbc.prep.observation_data(workdir)

In [ ]:
# Generate the clusters using the historical simulation data
rbc.cluster.generate(workdir)
assign_table = rbc.cluster.summarize(workdir, assign_table)
rbc.table.cache(workdir, assign_table)

In [ ]:
# Assign basins which are gauged
assign_table = rbc.assign.gauged(assign_table)

In [ ]:
# Assign basins by propagation from the gauges
assign_table = rbc.assign.propagation(assign_table)

In [ ]:
# Assign basins by clusters
assign_table = rbc.assign.clusters(assign_table)

In [ ]:
# Cache the assignments table with the updates
rbc.table.cache(workdir, assign_table)

In [5]:
# Generate GIS files so you can go explore your progress graphically
rbc.gis.clip_by_assignment(workdir, assign_table, drain_shape)
rbc.gis.clip_by_cluster(workdir, assign_table, drain_shape)